In [1]:
#!kaggle datasets download -d mushrafimunim/bn-corpus-prothom-alo

In [1]:
import pandas as pd 
import numpy  as np
from wordfreq import word_frequency ,top_n_list,get_frequency_dict

from symspellpy import SymSpell, Verbosity
from itertools import islice

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=4)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
sym_spell = SymSpell(max_dictionary_edit_distance=5, prefix_length=7)
dictionary_path = 'asr_bangla/prothom_alo_word_freq.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

In [3]:
df_wav2vec2 = pd.read_csv('symspell_nov.csv')

#df_prothom_alo = pd.read_csv('bangla_text_data.csv')

#df_prothom_alo= df_prothom_alo.dropna(how='any')
df_wav2vec2 = df_wav2vec2.dropna(how='any')
df_wav2vec2 = df_wav2vec2[df_wav2vec2.audio == 'noisy']

In [4]:
def wer(sen):
    sen = eval(sen)
    s = sen.count('S')
    d = sen.count('D')
    i = sen.count('I')
    return int(((s+d+i)*100)/len(sen))

In [5]:
eval(df_wav2vec2['2gram_label'][0])

['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']

In [6]:
df_wav2vec2['wer'] = df_wav2vec2['2gram_label'].parallel_apply(lambda x: wer(x))

In [7]:
df_wav2vec2 =  df_wav2vec2[df_wav2vec2.wer !=0]

In [8]:
df_wav2vec2 = df_wav2vec2[['sentence','wav2vec2','2gram',]]

In [9]:
#df_wav2vec2.rename(columns={'sentence':'refere'},inplace=True)

In [10]:
df_wav2vec2.iloc[1500].wav2vec2

'তিনি মোট পাঁচবার সংশদ নির্বাচনে অংশগ্রহণ করে চারবার বিজয়ী হন'

In [11]:
df_wav2vec2

,sentence,wav2vec2,2gram
2,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...
3,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি
4,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...
5,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে কারি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গারিকাপের শির...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গাড়ি কাপের শি...
6,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেতা পরিচালনা করেছেন রাখাল বন্ধুর পরিচালক ইবনে...
...,...,...,...
38375,পিছিয়ে থাকতে চাইনা,পিছিয়ে থাকতে চায় না,পিছিয়ে থাকতে চায় না
38376,তারপরেই রয়েছ,তারপরে রয়েছে,তারপরে রয়েছে
38381,একাধিকবার আহ্বান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন
38385,তাদের ছেলেমেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের


In [12]:
n=list(df_wav2vec2.sample(1).index)[0]

In [13]:
actual = df_wav2vec2['sentence'][n]
raw = df_wav2vec2['wav2vec2'][n]
arpa = df_wav2vec2['2gram'][n]

In [14]:
arpa

'কম্পিউটার ইঞ্জিনিয়ারিং ডিগ্রি নেন'

In [15]:
def masking(arpa):
    sen = arpa.split()
    for words in sen:
        suggestions = sym_spell.lookup(words, Verbosity.CLOSEST,max_edit_distance=0, include_unknown=True)
        for suggestion in suggestions:
            suggestion = str(suggestion)
            suggestion = suggestion.split(',')
            if int(suggestion[2]) == 0:
                mask = '[MASK]'
                sen = [mask if x == words else x for x in sen]
                break
    return " ".join(sen)


In [16]:
df_wav2vec2['masked_arpa'] = df_wav2vec2['2gram'].parallel_apply(lambda x: masking(str(x)))
df_wav2vec2['masked_wav2vec2'] = df_wav2vec2['wav2vec2'].parallel_apply(lambda x: masking(str(x)))

In [18]:
df_wav2vec2

,sentence,wav2vec2,2gram,masked_arpa,masked_wav2vec2
2,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...
3,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি
4,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,[MASK] এ সংগ্রহটি [MASK] ক্রিকেটে বিশ্ব রেকর্ড...
5,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে কারি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গারিকাপের শির...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গাড়ি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গাড়ি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে [MASK] শিরোপা...
6,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেতা পরিচালনা করেছেন রাখাল বন্ধুর পরিচালক ইবনে...,যেতা পরিচালনা করেছেন রাখাল বন্ধুর পরিচালক ইবনে...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...
...,...,...,...,...,...
38375,পিছিয়ে থাকতে চাইনা,পিছিয়ে থাকতে চায় না,পিছিয়ে থাকতে চায় না,পিছিয়ে থাকতে চায় না,পিছিয়ে থাকতে চায় না
38376,তারপরেই রয়েছ,তারপরে রয়েছে,তারপরে রয়েছে,তারপরে রয়েছে,তারপরে রয়েছে
38381,একাধিকবার আহ্বান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন
38385,তাদের ছেলেমেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের


In [19]:
mask = '[MASK]'

In [20]:
df_wav2vec2['2gram_masked_n'] = df_wav2vec2['masked_arpa'].parallel_apply(lambda x: str(x).split().count(mask))
df_wav2vec2['wav2vec2_masked_n'] = df_wav2vec2['masked_wav2vec2'].parallel_apply(lambda x: str(x).split().count(mask))

In [21]:
df_wav2vec2

,sentence,wav2vec2,2gram,masked_arpa,masked_wav2vec2,2gram_masked_n,wav2vec2_masked_n
2,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,0,0
3,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,0,0
4,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,[MASK] এ সংগ্রহটি [MASK] ক্রিকেটে বিশ্ব রেকর্ড...,0,2
5,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে কারি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গারিকাপের শির...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গাড়ি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে গাড়ি কাপের শি...,ফলশ্রুতিতে নাটাল দলকে পরাজিত করে [MASK] শিরোপা...,0,1
6,যেটা পরিচালনা করেছেন রাখাল বন্ধুর ই পরিচালক ইব...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,যেতা পরিচালনা করেছেন রাখাল বন্ধুর পরিচালক ইবনে...,যেতা পরিচালনা করেছেন রাখাল বন্ধুর পরিচালক ইবনে...,যেতা পরিচালনা করেছেন রাখাল বন্ধুরই পরিচালক ইবন...,0,2
...,...,...,...,...,...,...,...
38375,পিছিয়ে থাকতে চাইনা,পিছিয়ে থাকতে চায় না,পিছিয়ে থাকতে চায় না,পিছিয়ে থাকতে চায় না,পিছিয়ে থাকতে চায় না,0,0
38376,তারপরেই রয়েছ,তারপরে রয়েছে,তারপরে রয়েছে,তারপরে রয়েছে,তারপরে রয়েছে,0,0
38381,একাধিকবার আহ্বান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,একাধিকবার আহবান জানিয়েছেন,0,0
38385,তাদের ছেলেমেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,0,0


In [22]:
df_wav2vec2['wav2vec2_masked_n'].sum()

17925

In [37]:
df_wav2vec2 = df_wav2vec2[(df_wav2vec2.wav2vec2_masked_n != 0)].drop(['sentence','masked_arpa','2gram_masked_n'],axis=1)

In [46]:
df_wav2vec2['wlen'] = df_wav2vec2.wav2vec2.parallel_apply(lambda x: len(str(x).split()))
df_wav2vec2['2len'] = df_wav2vec2['2gram'].parallel_apply(lambda x: len(str(x).split()))

df_wav2vec2['diff'] = abs(df_wav2vec2['wlen'] - df_wav2vec2['2len'])
df_wav2vec2 = df_wav2vec2[df_wav2vec2['diff'] == 0 ]

In [49]:
df_wav2vec2 = df_wav2vec2.drop(['wlen','2len','diff'],axis=1)

In [64]:
df_wav2vec2[df_wav2vec2.masked_wav2vec2 == 1 ].wav2vec2_masked_n.sum()

0

In [150]:
arpa_x =  (arpa.split()[sen.index(mask)] if mask in sen else "")

id_mask = (sen.index(mask) if mask in sen else -100)

if id_mask != -100:
    if id_mask == 0:
        wav2vec2_x = raw.split()[0]
    if id_mask == (len(sen)-1):
        wav2vec2_x = raw.split()[-1]
    else:
        start_seq = arpa.split()[sen.index(mask)-1]
        end_seq = arpa.split()[sen.index(mask)+1]
        #needs work
        raw = raw.replace(start_seq,"*")
        raw = raw.replace(end_seq,"*")
        wav2vec2_x = raw.split("*")
        print(wav2vec2_x)
        #wav2vec2_x = wav2vec2_x[1].split()[0]
if id_mask == -100:
    wav2vec2_x = ""
    arpa_x = ""

In [358]:
actual

'জানুয়ারি তারিখ'

In [359]:
raw

'বাই জনয়ই দুহাজায় বারতারিক'

In [360]:
input_x

'বাই জন দুহাজার বার তারিক'

In [361]:
arpa_x

''

In [362]:
wav2vec2_x

''